Notebook permettant de générer les fichiers de schéma des fichiers parquet.

Storage Account Name: nom du storage account des containers bronze, silver et des fichiers de configuration.

In [0]:
%run ./Utils/COMMON

In [0]:
dbutils.widgets.dropdown("parquet_folder", "QAD_PARQUET", ["QAD_PARQUET", "GROUP_REFERENTIAL_PARQUET", "DELTA_MASTER_PARQUET","ORDER_MANAGEMENT_PARQUET"], "Parquet Folder")
dbutils.widgets.dropdown("config_folder", "QAD", ["QAD", "GROUP_REFERENTIAL", "DELTA_MASTER","ORDER_MANAGEMENT"], "config-files Folder")

parquet_folder: str = dbutils.widgets.get('parquet_folder')
config_folder: str = dbutils.widgets.get('config_folder')

errors_count = 0

In [0]:
from typing import List
import csv
import io

# Lecture du fichier de configuration.
tables = csv.DictReader(io.StringIO(dbutils.fs.head(f"dbfs:/mnt/dlk/{CONFIG_CONTAINER}/{config_folder}/params.csv")), delimiter=';')

params = []
for table in tables:
  table_name = table.get('output')
  bronze_location: str = f"dbfs:/mnt/dlk/{BRONZE_CONTAINER}/{parquet_folder}/{table_name}/"
  schema_path: str = f"dbfs:/mnt/dlk/{CONFIG_CONTAINER}/{config_folder}/TableSchemas/{table_name}.json"
  
  params.append({'input_file_path': bronze_location,
                 'schema_path': schema_path, 
                 'folder_partition': 'year=*/month=*/day=*/time=*'
                })

In [0]:
import multiprocessing as mp
manager = mp.Manager()
pool = mp.pool.ThreadPool(5)

results = pool.map(lambda param: LIN_run_notebook('./Jobs/PARQUET_TO_SCHEMA', param), params)
errors_count = sum(results)

# Si un chargement est en erreur, on leve une exception pour que le notebook soit en erreur. 
if errors_count < 0:
  raise ValueError("Unable to load all files.")